# 文本情感分析：使用 LSTM

我們將使用 LSTM 方法來做情感(情緒)的分析，資料集是著名的資料集電影評論資料集 IMDB dataset.
期望達到不錯的準確度 84% 以上


## 準備資料

torchtext 包含以下 components：

Field : 主要包含以下數據預處理的配置信息：指定分詞方法、是否轉成小寫、起始符號、以及字典等等。

Dataset : 用於下載數據，也提供 splits 方法可以同時下載訓練資料、驗證資料和測試資料。

Iterator : 數據讀取的迭代器，可以支持 batch

我們定義 SEED、TEXT 和 LABEL 三個變數來隨機把資料集分割成 train/valid/test 三個資料集。



In [1]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

import spacy
# nlp = spacy.load("en_core_web_sm")
TEXT = data.Field(tokenize='moses', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

## 下載並讀取資料

torchtext 的 datasets 集合裡面就有 IMDB 資料，直接就可以讀取訓練以及測試資料了。

In [2]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)


## 從訓練資料裡面切割驗證資料

從訓練資料裡面抓取一些資料當作 validation set

In [3]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

# 建立字典

接下來是使用預訓練的 word embeddings。只要呼叫 TorchText 的 build_vocab 就可以把所有的文字向量化, 我們使用的是 "glove.6B.100d" 的向量，glove 是一個用來計算詞向量的演算法。6B 是指這些詞向量是用了60億個tokens訓練出來的，而 100d 是指每一個向量的維度是 100。

In [4]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

#建立Iterator

Iterator 是 torchtext 到模型的輸出，提供了對數據的打亂、排序等等處理方法。可以動態修改 batch size這裡使用 splits method 來同時輸出訓練集、驗證集以及測試集。

如果有 GPU 的話則使用 cuda 來做運算。

`sort_within_batch = True` 是表示在每一個 batch 裡面的 tensors 是依照長度排序的。

In [5]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

# 建立 LSTM 模型
我們將使用 pytorch 內建的 RNN 架構是 LSTM (Long Short-Term Memory)模型。它的公式如下：

$(h_t, c_t) = \text{LSTM}(x_t, h_t, c_t)$


步驟解釋：

1. 在模型裡，每個詞會先通過 embedding layer 的到特徵向量
2. 然後我們使用 LSTM 對特徵序列進一步編碼得到序列信息。
3. 將編碼後的序列信息通過全連接層(Fully connectivity layer)得到輸出。




In [6]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu())
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)


# LSTM 模型參數說明

1. vocab_size: 輸入層的維度(input dim)
2. embedding_dim: 詞向量的維度, 我們使用的是 glove.6B.100d, 所以這裡 embedding_dim 是 100
3. hidden_dim: the size of the hidden states
4. output_dim: 輸出層的維度
5. n_layers: 有幾層全連結層
6. bidirectional: 是否使用雙向 RNN
7. dropout： dropout 的比例
8. pad_idx: token <pad> 的 index


In [7]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

印出我們模型的參數量

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters



檢查 embedding 的字典大小以及 embedding 的維度

_**[vocab size, embedding dim]**_ 


In [9]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


用 pre-trained embeddings 來當作 `embedding` 層的初始化參數


In [10]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.6181,  0.1835,  0.5297,  ...,  0.2501, -0.5837, -0.0177],
        [-0.2372, -0.0055, -0.1402,  ..., -0.0492,  0.2670, -0.5836],
        [-0.4868,  0.9502, -0.5040,  ...,  0.0096, -0.1862,  0.3399]])

因為`<unk>` and `<pad>`是沒有在 pre-trained 的詞裡面，所以要把 `<unk>` 和 `<pad>`的 初始 embedding 權重都變成 0

In [11]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.6181,  0.1835,  0.5297,  ...,  0.2501, -0.5837, -0.0177],
        [-0.2372, -0.0055, -0.1402,  ..., -0.0492,  0.2670, -0.5836],
        [-0.4868,  0.9502, -0.5040,  ...,  0.0096, -0.1862,  0.3399]])


We can now see the first two rows of the embedding weights matrix have been set to zeros. As we passed the index of the pad token to the `padding_idx` of the embedding layer it will remain zeros throughout training, however the `<unk>` token embedding will be learned.

## 訓練模型

使用優化器 Adam

In [12]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

使用 BCEWithLogitsLoss 當作 Loss Function

In [13]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

實作計算計算準確度的函式



In [14]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

# 訓練函式

In [15]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# 測試模型的方法


In [16]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

查看訓練進度以及花費的時間

In [17]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

開始訓練

In [18]:
N_EPOCHS = 5


best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 566m 3s
	Train Loss: 0.688 | Train Acc: 54.97%
	 Val. Loss: 0.671 |  Val. Acc: 59.97%
Epoch: 02 | Epoch Time: 534m 41s
	Train Loss: 0.689 | Train Acc: 53.36%
	 Val. Loss: 0.686 |  Val. Acc: 53.92%
Epoch: 03 | Epoch Time: 571m 50s
	Train Loss: 0.664 | Train Acc: 59.59%
	 Val. Loss: 0.503 |  Val. Acc: 75.94%
Epoch: 04 | Epoch Time: 532m 47s
	Train Loss: 0.575 | Train Acc: 70.39%
	 Val. Loss: 0.444 |  Val. Acc: 80.44%
Epoch: 05 | Epoch Time: 545m 16s
	Train Loss: 0.447 | Train Acc: 79.69%
	 Val. Loss: 0.408 |  Val. Acc: 81.88%


測試模型的準確度

In [19]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.414 | Test Acc: 81.39%


## Demo 函式

最後我們來建立一個 Demo 的函式讓使用者可以輸入任意的句子來看看模型是否可以正確的做好情緒分類。

In [20]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

# Demo

負面情緒的句子

In [21]:
predict_sentiment(model, "This film is terrible")

0.04404826462268829

# Demo 
正面情緒的句子

In [22]:
predict_sentiment(model, "This film is great")

0.9438639879226685

## 參考資料

torchtext 入門

https://zhuanlan.zhihu.com/p/31139113